In [145]:
import pandas as pd

In [146]:
import pyodbc

In [147]:
server = '157.119.230.120,4071'
database = 'PowerBI_LISV2'
username = 'sa'
password = 'Y0m@Sql.!.123'

conn_str = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
"""

In [148]:
conn = pyodbc.connect(conn_str)
query = """
WITH Attendance_CTE AS (
    SELECT 
        A.UserName,
		A.FullName,
		A.Region,
		A.Designation,
        CASE
            WHEN A.Status = 'Present' THEN 'P'
            WHEN A.Status = 'Absent' THEN 'A'
            WHEN A.Status = 'Leave' THEN 'L'
            WHEN A.Status = 'Compensatory Off' THEN 'CO'
            WHEN A.Status IN ('Week Off', 'WeekOff') THEN 'WO'
            WHEN A.Status = 'Meeting' THEN 'M'
			WHEN A.Status = 'Training' THEN 'T'
			WHEN A.Status = 'National Hoilday' THEN 'H'
            ELSE 'Condition not found'
        END AS AttStatus,
        FORMAT(TRY_CONVERT(date, A.[Date], 105), 'dd') AS DayNum
    FROM PowerBI_LISV2.[GCPLFoodsManagement].ISPAttendanceMasterExport AS A
    LEFT JOIN PowerBI_LISV2.[GCPLFoodsManagement].UserMaster AS M
        ON A.UserName = M.UserName
    WHERE 
        MONTH(TRY_CONVERT(date, A.[Date], 105)) = 09
        AND M.Status = 'True'
        AND (A.UserName LIKE '%ISP%' or A.UserName LIKE '%MR%')
),

-- Pivoted daily attendance
Pivoted AS (
    SELECT *
    FROM Attendance_CTE
    PIVOT (
        MAX(AttStatus)
        FOR DayNum IN (
            [01],[02],[03],[04],[05],[06],[07],[08],[09],[10],
            [11],[12],[13],[14],[15],[16],[17],[18],[19],[20],
            [21],[22],[23],[24],[25],[26],[27],[28],[29],[30],[31]
        )
    ) AS P
),
-- Summary counts per user
Summary AS (
    SELECT 
        UserName,
        SUM(CASE WHEN AttStatus = 'P' THEN 1 ELSE 0 END) AS P_Count,
        SUM(CASE WHEN AttStatus = 'A' THEN 1 ELSE 0 END) AS A_Count,
        SUM(CASE WHEN AttStatus = 'L' THEN 1 ELSE 0 END) AS L_Count,
		SUM(CASE WHEN AttStatus = 'CO' THEN 1 ELSE 0 END) AS CO_Count,
		SUM(CASE WHEN AttStatus = 'WO' THEN 1 ELSE 0 END) AS WO_Count,
		SUM(CASE WHEN AttStatus = 'H' THEN 1 ELSE 0 END) AS H_Count,
		SUM(CASE WHEN AttStatus = 'T' THEN 1 ELSE 0 END) AS T_Count,
		SUM(CASE WHEN AttStatus = 'M' THEN 1 ELSE 0 END) AS M_Count
    FROM Attendance_CTE
    GROUP BY UserName
)

-- Final join to combine both outputs
SELECT 
    P.UserName,
	P.FullName,
	P.Region,
	P.Designation,
    P.[01], P.[02], P.[03], P.[04], P.[05], P.[06], P.[07], P.[08], P.[09], P.[10],
    P.[11], P.[12], P.[13], P.[14], P.[15], P.[16], P.[17], P.[18], P.[19], P.[20],
    P.[21], P.[22], P.[23], P.[24], P.[25], P.[26], P.[27], P.[28], P.[29], P.[30], P.[31],
    S.P_Count,
    S.A_Count,
	S.L_Count,
    S.WO_Count,
	S.M_Count,
	S.CO_Count
FROM Pivoted P
LEFT JOIN Summary S
    ON P.UserName = S.UserName;

                
"""

In [149]:
query2 = """

    SELECT A.*,
        A.UserName,
        A.FullName,
        A.Region,
        A.Designation,
        A.Status,
        CASE
            WHEN A.Status = 'Present' THEN 'P'
            WHEN A.Status = 'Absent' THEN 'A'
            WHEN A.Status = 'Leave' THEN 'L'
            WHEN A.Status = 'Compensatory Off' THEN 'CO'
            WHEN A.Status IN ('Week Off', 'WeekOff') THEN 'WO'
            WHEN A.Status = 'Meeting' THEN 'M'
            WHEN A.Status = 'Training' THEN 'T'
            WHEN A.Status IN ('National Hoilday','Hoilday') THEN 'H'
            WHEN A.Status = 'Hoilday' THEN 'H'
            ELSE 'NA'
        END AS AttStatus,
        DAY(TRY_CONVERT(date, A.[Date], 105)) AS DayNum
    FROM PowerBI_LISV2.[GCPLFoodsManagement].ISPAttendanceMasterExport AS A
    LEFT JOIN PowerBI_LISV2.[GCPLFoodsManagement].UserMaster AS M
        ON A.UserName = M.UserName
    WHERE TRY_CONVERT(date, A.[Date], 105) >= 
    CASE 
        WHEN DAY(GETDATE()) = 1 
        THEN DATEFROMPARTS(YEAR(DATEADD(MONTH, -1, GETDATE())), MONTH(DATEADD(MONTH, -1, GETDATE())), 1)
        ELSE DATEFROMPARTS(YEAR(GETDATE()), MONTH(GETDATE()), 1)
    END
        AND TRY_CONVERT(date, A.[Date], 105) < 
    CASE 
        WHEN DAY(GETDATE()) = 1 
        THEN DATEADD(MONTH, 1, DATEFROMPARTS(YEAR(DATEADD(MONTH, -1, GETDATE())), MONTH(DATEADD(MONTH, -1, GETDATE())), 1))
        ELSE DATEADD(MONTH, 1, DATEFROMPARTS(YEAR(GETDATE()), MONTH(GETDATE()), 1))
    END


        AND M.Status = 'True'
        AND (A.UserName LIKE '%ISP%' OR A.UserName LIKE '%MR%')


    
"""

In [150]:
Attendence_df = pd.read_sql(query2, conn)

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_16732\3254521587.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Attendence_df = pd.read_sql(query2, conn)


In [151]:
Attendence_df=Attendence_df.fillna('')

In [152]:
Attendence_df.head()

ISPAttendanceMasterExportId Month  Year Region        State     City  \
0                       123273     6  2025   East  WEST BENGAL  Kolkata   
1                       127482     7  2025   East  WEST BENGAL  Kolkata   
2                       135309     9  2025   East  WEST BENGAL  Kolkata   
3                       135532     9  2025   East  WEST BENGAL  Kolkata   
4                       135665     9  2025   East  WEST BENGAL  Kolkata   

  RSMCode BranchCode ASMCode SOCode  ... EmployeeStatus     PositionCode  \
0                                    ...         Active  LISF-GFD-WB-113   
1                                    ...         Active  LISF-GFD-WB-113   
2                                    ...         Active  LISF-GFD-WB-113   
3                                    ...         Active  LISF-GFD-WB-113   
4                                    ...         Active  LISF-GFD-WB-113   

  PositionCodeStatus      UserName             FullName Region Designation  \
0             Active  GCPLF-ISP-01  Suryakanta Pramanik   East    Promoter   
1             Active  GCPLF-ISP-01  Suryakanta Pramanik   East    Promoter   
2             Active  GCPLF-ISP-01  Suryakanta Pramanik   East    Promoter   
3             Active  GCPLF-ISP-01  Suryakanta Pramanik   East    Promoter   
4             Active  GCPLF-ISP-01  Suryakanta Pramanik   East    Promoter   

     Status AttStatus DayNum  
0   Present         P      6  
1   Present         P      7  
2   Present         P      1  
3  Week Off        WO      2  
4     Leave         L      3  

[5 rows x 46 columns]

In [153]:
Attendence_df.to_csv("gcplAttendence_df.csv")